In [315]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
# update
train = pd.read_csv('./train.csv', parse_dates= ['date'])
train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [316]:
test = pd.read_csv('./test.csv', parse_dates= ['date'])
test

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [317]:
print('Min date from train set : ' + str(train['date'].min().date()))
print('Max date from train set : ' + str(train['date'].max().date()))

Min date from train set : 2013-01-01
Max date from train set : 2017-12-31


In [318]:
lag_size = (test['date'].max().date() - train['date'].max().date()).days 
print('Max date from train set : ' + str(train['date'].max().date()) )
print('Max date from test set : ' +str(test['date'].max().date()) )
print('Forecast lag size', lag_size)

Max date from train set : 2017-12-31
Max date from test set : 2018-03-31
Forecast lag size 90


In [319]:
dates = train.groupby('date', as_index = False)['sales'].sum()
stores = train.groupby(['store', 'date'], as_index = False)['sales'].sum()
items = train.groupby(['item', 'date'], as_index = False)['sales'].sum()

dates

,date,sales
0,2013-01-01,13696
1,2013-01-02,13678
2,2013-01-03,14488
3,2013-01-04,15677
4,2013-01-05,16237
...,...,...
1821,2017-12-27,20378
1822,2017-12-28,21885
1823,2017-12-29,23535
1824,2017-12-30,24988


In [320]:
stores

,store,date,sales
0,1,2013-01-01,1316
1,1,2013-01-02,1264
2,1,2013-01-03,1305
3,1,2013-01-04,1452
4,1,2013-01-05,1499
...,...,...,...
18255,10,2017-12-27,2221
18256,10,2017-12-28,2429
18257,10,2017-12-29,2687
18258,10,2017-12-30,2742


In [321]:
items

,item,date,sales
0,1,2013-01-01,133
1,1,2013-01-02,99
2,1,2013-01-03,127
3,1,2013-01-04,145
4,1,2013-01-05,149
...,...,...,...
91295,50,2017-12-27,511
91296,50,2017-12-28,587
91297,50,2017-12-29,596
91298,50,2017-12-30,612


In [322]:
!pip3 install plotly
# !pip3 install "nbformat>=4.2.4.0" notebook

import plotly.graph_objs as go 
from plotly.offline import iplot

plot_dates = go.Scatter(x = dates['date'], y = dates['sales'])
layout = go.Layout(title = 'Daily Sales', xaxis = dict(title= 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = [plot_dates], layout = layout)

iplot(fig)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [324]:
plot_stores = [] 
for store in stores['store'].unique(): 
    c_store = stores[(stores['store'] == store)]
    plot_stores.append(go.Scatter(x=c_store['date'], y = c_store['sales'], name = ('Store ' + str(store))))
layout = go.Layout(title = 'Store daily sales', xaxis = dict(title='Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = plot_stores, layout = layout)

iplot(fig)

In [325]:
plot_items = []
for item in items ['item'].unique():
    c_items = items[(items['item'] == item)]
    plot_items.append(go.Scatter(x = c_items['date'], y = c_items['sales'], name = ('Item ' + str(item))))
layout = go.Layout(title = 'Item daily sales', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = plot_items, layout = layout)

iplot(fig)

In [326]:
indexes_train = []
for i in range(train.store.nunique()*train.item.nunique()) :
    indexes_train = np.append(indexes_train, np.linspace(0, train.date.nunique()-1, train.date.nunique(), dtype = int))
    i = +1
indexes_train

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.823e+03, 1.824e+03,
       1.825e+03])

In [327]:
train["indexes"] = indexes_train
train

,date,store,item,sales,indexes
0,2013-01-01,1,1,13,0.0
1,2013-01-02,1,1,11,1.0
2,2013-01-03,1,1,14,2.0
3,2013-01-04,1,1,13,3.0
4,2013-01-05,1,1,10,4.0
...,...,...,...,...,...
912995,2017-12-27,10,50,63,1821.0
912996,2017-12-28,10,50,59,1822.0
912997,2017-12-29,10,50,74,1823.0
912998,2017-12-30,10,50,62,1824.0


In [331]:
indexes_test = []
test["indexes"] = indexes_test 
test

ValueError: Length of values (0) does not match length of index (45000)